In [1]:
import pandas as pd
import numpy as np
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

In [2]:
query = """ 
CALL gds.graph.drop('Got2') YIELD graphName;
"""

query2 = """
CALL gds.graph.drop('Got3') YIELD graphName;
"""

graph.run(query)
graph.run(query2)

ClientError: [Procedure.ProcedureCallFailed] Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `Got2` does not exist on database `neo4j`. It might exist on another database.

## Season 5&6

In [3]:
query = """ 
CALL gds.graph.project(
    'Got2',
    {
        Person_5: {properties: 'seed' },
        Person_6: {properties: 'seed' }
    },
    {
        INTERACTS_5: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_6: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
)
"""

graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person_6: {label: 'Person_6', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_5: {label: 'Person_5', properties: {seed: {defaultValue: null, property: 'seed'}}}}","{INTERACTS_6: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_6', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_5: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_5', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Got2,188,1892,118


In [4]:
# The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.beta.leiden.write.estimate('Got2', { writeProperty: 'community_S56' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)


nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
188,1892,610416,615624,[596 KiB ... 601 KiB]


In [8]:
# The following will run the algorithm and stream results:
query = """
CALL gds.beta.leiden.stream('Got2', {seedProperty: 'seed', relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
a = graph.run(query).to_data_frame()


In [23]:
a.groupby('communityId')['label'].apply(list)


communityId
0     [Aegon, Aemon, Alliser, Areo, Arthur (child), ...
11    [Aerys, Barristan, Belicho, Daario, Daenerys, ...
26    [Arthur, Benjen, Bran, Hodor, Howland, Leaf, L...
29                                             [Wolkan]
32    [Aeron, Balon, Brienne, Catelyn, Cerwyn, Euron...
44    [Beric, Flynn, Gatins, Lem, Morgan, Ray, Sando...
Name: label, dtype: object

In [16]:
a.groupby('communityId')['label'].apply(list)[11]  # Group with Daerneys Targaryen


['Aerys',
 'Barristan',
 'Belicho',
 'Daario',
 'Daenerys',
 'Dom',
 'Dosh Khaleen Leader',
 'Drogo',
 'Grey Worm',
 'Hizdahr',
 'Jorah',
 'Khal Moro',
 'Kinvara',
 'Kraznys',
 'Lhazareen Woman',
 'Malko',
 'Missandei',
 'Mossador',
 'Owner',
 'Quick',
 'Razdal',
 'Red Priest',
 'Rhaegar',
 'Shae',
 'Strong',
 'Strumpet',
 'Tyrion',
 'Vala',
 'Varys',
 'Yezzan']

In [14]:
a.groupby('communityId')['label'].apply(list)[26] # Group in the far north

['Arthur',
 'Benjen',
 'Bran',
 'Hodor',
 'Howland',
 'Leaf',
 'Lyanna',
 'Lynanna',
 'Meera',
 'Ned',
 'Night King',
 'Old Nan',
 'Rickard Stark',
 'Rodrik',
 'Three Eyed Raven']

In [19]:
a.groupby('communityId')['label'].apply(list)[32] # Group in the north


['Aeron',
 'Balon',
 'Brienne',
 'Catelyn',
 'Cerwyn',
 'Euron',
 'Lysa',
 'Myranda',
 'Osha',
 'Petyr',
 'Podrick',
 'Ramsay',
 'Renly',
 'Rickard Karstark',
 'Rickon',
 'Robin',
 'Roose',
 'Rooses Son',
 'Royce',
 'Sansa',
 'Selwyn',
 'Smalljon',
 'Theon',
 'Walda',
 'Wolkan',
 'Yara',
 'Yohn']

In [22]:
a.groupby('communityId')['label'].apply(list)[0] # Group in Kings Landing


['Aegon',
 'Aemon',
 'Alliser',
 'Areo',
 'Arthur (child)',
 'Arya',
 'Bianca',
 'Black Walder',
 'Bobono',
 'Bowen',
 'Brand',
 'Brian',
 'Bronn',
 'Brynden',
 'Camello',
 'Cersei',
 'Citadel Maester',
 'Clarenzo',
 'Davos',
 'Denys',
 'Derryk',
 'Dickon',
 'Dim Dalba',
 'Doran',
 'Dorne Maester',
 'Dornish Rider',
 'Eddison',
 'Edmure',
 'Edmures Son',
 'Ellaria',
 'Falyse',
 'Gilly',
 'Glover',
 'Gregor',
 'Grenn',
 'Guard Captain',
 'Harald',
 'High Septon',
 'High Sparrow',
 'Hoster',
 'Izembaro',
 'Jaime',
 'Janos',
 'Jaqen',
 'Jeor',
 'Joffrey',
 'Jon',
 'Karsi',
 'Kevan',
 'Lady Crane',
 'Lancel',
 'Little Sam',
 'Loboda',
 'Lollys',
 'Loras',
 'Lord Weebly',
 'Lord of Bones',
 'Lothar',
 'Lyanna Mormont',
 'Mace',
 'Madame',
 'Maggy',
 'Magnar',
 'Mance',
 'Manderly',
 'Margaery',
 'Melara',
 'Melessa',
 'Melisandre',
 'Meryn',
 'Minisa',
 'Mormont Maester',
 'Mountain',
 'Myrcella',
 'Nymeria',
 'Obara',
 'Oberyn',
 'Olenna',
 'Olly',
 'Olyvar',
 'Othell',
 'Pycelle',
 'Pyp',

In [6]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.beta.leiden.stats('Got2')
YIELD communityCount
"""
graph.run(query)


communityCount
8


In [7]:
# The following run the algorithm, and write back results:
query = """
CALL gds.beta.leiden.write('Got2', { writeProperty: 'community_S56' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)


communityCount,modularity,modularities
8,0.6069910472044303,"[0.5699412011853627, 0.5964358666064747, 0.6069910472044303]"


In [24]:
# Degree
query = """
CALL gds.degree.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,SANSA,62.0
1,CERSEI,58.0
2,JON,56.0
3,TYRION,45.0
4,JAIME,41.0


In [25]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'Got2',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,JON,1150.0
1,CERSEI,924.0
2,TYRION,922.0
3,SANSA,913.0
4,JAIME,735.0


In [26]:
# Eigenvector
query = """
CALL gds.eigenvector.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,SANSA,0.326439
1,CERSEI,0.270071
2,JON,0.250806
3,LITTLEFINGER,0.201531
4,STANNIS,0.199998


In [27]:
# Pagerank
query = """
CALL gds.pageRank.stream('Got2',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,page,score
0,JON,4.703863
1,SANSA,4.685592
2,CERSEI,4.273345
3,TYRION,3.870091
4,JAIME,3.251898


In [28]:
# Betweenness
query = """
CALL gds.betweenness.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,SANSA,3249.913507
1,JON,2809.363537
2,TYRION,2453.560830
3,JAIME,2244.448242
4,CERSEI,2073.401141


<img src="https://upload.wikimedia.org/wikipedia/en/5/59/Game_of_Thrones_Season_5.png"></img>
<img src="https://upload.wikimedia.org/wikipedia/en/d/d1/Game_of_Thrones_Season_6.jpeg"></img>

Season 5 and 6 are two of the most important seasons in the series. These are the seasons where the characters move to their allies and major power struggles begin. Sansa, Jon and Cersie are seen to be the most important people in these seasons. They top all the 5 centrality categories. Several important events involving these characters are seen in these seasons. Jon gets killed and rises back from the dead, Sansa wins the battle of Bastards for Jon and Cersei demolishes all her enemies in the Kings Landing. These are the seasons where the show picks up pace and is on a cruise to the inevitable War against the White Walkers. 

## Season 7&8

In [13]:
query = """ 
CALL gds.graph.project(
    'Got3',
    {
        Person_7: {properties: 'seed' },
        Person_8: {properties: 'seed' }
    },
    {
        INTERACTS_7: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        },
        INTERACTS_8: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
)
"""

graph.run(query)


nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{Person_7: {label: 'Person_7', properties: {seed: {defaultValue: null, property: 'seed'}}}, Person_8: {label: 'Person_8', properties: {seed: {defaultValue: null, property: 'seed'}}}}","{INTERACTS_8: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_8', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}, INTERACTS_7: {orientation: 'UNDIRECTED', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACTS_7', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Got3,104,1932,28


In [14]:
# The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.beta.leiden.write.estimate('Got3', { writeProperty: 'community_S78' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)


nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
104,1932,588224,591080,[574 KiB ... 577 KiB]


In [15]:
# The following will run the algorithm and stream results:
query = """
CALL gds.beta.leiden.stream('Got3', {seedProperty: 'seed', relationshipWeightProperty: 'weight'})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query).to_data_frame()


,label,communityId
0,Aegon,1
1,Aerys,1
2,Alanna,6
3,Alton,7
4,Alys,6
...,...,...
99,Willa,0
100,William,7
101,Wolkan,6
102,Yara,7


In [16]:
# The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.beta.leiden.stats('Got3')
YIELD communityCount
"""
graph.run(query)


communityCount
5


In [17]:
# The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.beta.leiden.mutate('Got3', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)


communityCount,modularity,modularities
5,0.22507222800903595,"[0.21793944420868533, 0.22339619527710267, 0.22507222800903595]"


In [18]:
# The following run the algorithm, and write back results:
query = """
CALL gds.beta.leiden.write('Got3', { writeProperty: 'community_S78' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)


communityCount,modularity,modularities
5,0.22734837476263348,"[0.21012349489260101, 0.22486647463017972, 0.22734837476263348]"


In [29]:
# Degree
query = """
CALL gds.degree.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,JON,79.0
1,TYRION,75.0
2,DAENERYS,75.0
3,SANSA,64.0
4,DAVOS,64.0


In [30]:
# Weighted Degree
query = """
CALL gds.degree.stream(
   'Got3',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,JON,1882.0
1,TYRION,1755.0
2,DAENERYS,1576.0
3,SANSA,1119.0
4,JAIME,1107.0


In [31]:
# Eigenvector
query = """
CALL gds.eigenvector.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,JON,0.254100
1,TYRION,0.244820
2,DAENERYS,0.235475
3,DAVOS,0.233078
4,BRIENNE,0.216607


In [32]:
# Pagerank
query = """
CALL gds.pageRank.stream('Got3',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,page,score
0,JON,3.667777
1,DAENERYS,3.509703
2,TYRION,3.299509
3,SANSA,3.073163
4,ARYA,2.865504


In [33]:
# Betweenness
query = """
CALL gds.betweenness.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query).to_data_frame()


,name,score
0,JON,542.696894
1,ARYA,513.175492
2,SAM,512.808655
3,SANSA,497.389139
4,DAENERYS,471.243228


<img src = "https://upload.wikimedia.org/wikipedia/en/9/92/Game_of_Thrones_Season_7.png"></img>
<img src="https://upload.wikimedia.org/wikipedia/en/e/e0/Game_of_Thrones_Season_8.png"></img>

Season 7 and 8, the final seasons of the series is full of twists and turns. We see the North and South team up to defeat the White Walkers, the battle for Kings Landing and finally Jon Snow killing Daerneys. The least expected Bran becomes the King and Tyrion his Hand. The North remains an independent Domain. The Centrality metrics paint the clear picture of Jon being the most important character in the last two seasons. He was indispensible in uniting the North and the South to fight against the White Walkers. The Starks are on a whole seen to rise up in the centrality ranks ascertaining the fact that Game of thrones is indeed the story of the Starks. Daerneys also rises in the ranks owing to her contribution to the storyline.